## 6.1从全连接层到卷积层

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [2]:
class CNNNet(nn.Module):
    def __init__(self):
        super(CNNNet,self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3,out_channels=16,kernel_size=5,stride=1)
        self.pool1 = nn.MaxPool2d(kernel_size=2,stride=2)
        self.conv2 = nn.Conv2d(in_channels=16,out_channels=36,kernel_size=3,stride=1)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(1296,128)
        self.fc2 = nn.Linear(128,10)      

    def forward(self,x):
        x=self.pool1(F.relu(self.conv1(x)))
        x=self.pool2(F.relu(self.conv2(x)))
        #print(x.shape)
        x=x.view(-1,36*6*6)
        x=F.relu(self.fc2(F.relu(self.fc1(x))))
        return x

net = CNNNet()
net=net.to(device)

## 6.2 卷积层
1、用PyTorch自定义卷积运算

In [3]:
def cust_conv2d(X, K): 
    """实现卷积运算"""
    #获取卷积核形状
    h, w = K.shape
    #初始化输出值Y
    Y = torch.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))
    #实现卷积运算
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i:i + h, j:j + w] * K).sum()
    return Y

In [4]:
X = torch.tensor([[1.0,1.0,1.0,0.0,0.0], [0.0,1.0,1.0,1.0,0.0], 
                  [0.0,0.0,1.0,1.0,1.0],[0.0,0.0,1.0,1.0,0.0],[0.0,1.0,1.0,0.0,0.0]])
K = torch.tensor([[1.0, 0.0,1.0], [0.0, 1.0,0.0],[1.0, 0.0,1.0]])
cust_conv2d(X, K)

tensor([[4., 3., 4.],
        [2., 4., 3.],
        [2., 3., 4.]])

### 6.2.2 学习卷积核

In [5]:
X = torch.tensor([[10.,10.,10.,0.0,0.0,0.0], [10.,10.,10.,0.0,0.0,0.0], [10.,10.,10.,0.0,0.0,0.0],
                  [10.,10.,10.,0.0,0.0,0.0],[10.,10.,10.,0.0,0.0,0.0],[10.,10.,10.,0.0,0.0,0.0]])
Y = torch.tensor([[0.0, 30.0,30.0,0.0], [0.0, 30.0,30.0,0.0],[0.0, 30.0,30.0,0.0],[0.0, 30.0,30.0,0.0]])

In [6]:
# 构造一个二维卷积层，它具有1个输出通道和形状为（3，3）的卷积核
conv2d = nn.Conv2d(1,1, kernel_size=(3, 3), bias=False)

# 这个二维卷积层使用四维输入和输出格式（批量大小、通道、高度、宽度），
# 其中批量大小和通道数都为1
X = X.reshape((1, 1, 6, 6))
Y = Y.reshape((1, 1, 4, 4))
lr = 0.001 # 学习率

#定义损失函数
loss_fn = torch.nn.MSELoss()
for i in range(400):
    Y_pre = conv2d(X)
    #loss = (Y_pre - Y) ** 2
    loss=loss_fn(Y_pre,Y)
    conv2d.zero_grad()
    loss.backward()
    # 迭代卷积核
    conv2d.weight.data[:] -= lr * conv2d.weight.grad
    if (i + 1) % 100 == 0:
        print(f'epoch {i+1}, loss {loss.sum():.4f}')

epoch 100, loss 0.0001
epoch 200, loss 0.0000
epoch 300, loss 0.0000
epoch 400, loss 0.0000


In [7]:
conv2d.weight.data.reshape((3,3))

tensor([[ 1.0961, -0.1821, -0.8602],
        [ 1.1862,  0.0869, -1.1806],
        [ 0.7178,  0.0952, -0.9593]])

### 6.2.5 多通道上的卷积

1、多输入

In [8]:
def corr2d_mutil_in(X,K):
    h,w = K.shape[1],K.shape[2]
    value = torch.zeros(X.shape[0] - h + 1,X.shape[1] - w + 1)
    for x,k in zip(X,K):
        value = value + cust_conv2d(x,k)
    return value

In [17]:
X = torch.tensor([[[1.,0.,1,0.,2.],[1,1,3,2,1],[1,1,0,1,1],[2,3,2,1,3],[0,2,0,1,0]],
                  [[1.,0.,0,1.,0.],[2,0,1,2,0],[3,1,1,3,0],[0,3,0,3,2],[1,0,3,2,1]],
                  [[2.,0.,1.,2.,1.],[3,3,1,3,2],[2,1,1,1,0],[3,1,3,2,0],[1,1,2,1,1]]])
K = torch.tensor([[[0.0,1.0,0.0],[0.0,0.0,2.0],[0.0,1.0,0.0]],
                  [[2.0,1.0,0.0],[0.0,0.0,0.0],[0.0,3.0,0.0]],
                  [[1.0,0.0,0.0],[1.0,0.0,0.0],[0.0,0.0,2.0]]])

In [18]:
print(K.shape)
corr2d_mutil_in(X,K)

torch.Size([3, 3, 3])


tensor([[19., 13., 15.],
        [28., 16., 20.],
        [23., 18., 25.]])

2、多输出通道

In [21]:
def corr2d_multi_in_out(X,K):
    return torch.stack([corr2d_mutil_in(X,k) for k in K])

3、1x1卷积核

In [23]:
X = torch.tensor([[[1,2,3],[4,5,6],[7,8,9]],
                  [[1,1,1],[1,1,1],[1,1,1]],
                  [[2,2,2],[2,2,2],[2,2,2]]])
 
K = torch.tensor([[[[1]],[[2]],[[3]]],
                  [[[4]],[[1]],[[1]]],
                  [[[5]],[[3]],[[3]]]])
print(K.shape)

torch.Size([3, 3, 1, 1])


In [24]:
def corr2d_multi_in_out(X,K):
    return torch.stack([corr2d_mutil_in(X,k) for k in K])
 
corr2d_multi_in_out(X,K)

tensor([[[ 9., 10., 11.],
         [12., 13., 14.],
         [15., 16., 17.]],

        [[ 7., 11., 15.],
         [19., 23., 27.],
         [31., 35., 39.]],

        [[14., 19., 24.],
         [29., 34., 39.],
         [44., 49., 54.]]])

<h3>6.3池化层</h3>

### 6.3.1局部池化

In [19]:
# 池化窗口为正方形 size=3, stride=2
m1 = nn.MaxPool2d(3, stride=2)
# 池化窗口为非正方形
m2 = nn.MaxPool2d((3, 2), stride=(2, 1))
input = torch.randn(20, 16, 50, 32)
output = m2(input)
print(output.shape)

torch.Size([20, 16, 24, 31])


### 6.3.2全局池化

In [20]:
# 输出大小为5x7
m = nn.AdaptiveMaxPool2d((5,7))
input = torch.randn(1, 64, 8, 9)
output = m(input)
# t输出大小为正方形 7x7 
m = nn.AdaptiveMaxPool2d(7)
input = torch.randn(1, 64, 10, 9)
output = m(input)
# 输出大小为 10x7
m = nn.AdaptiveMaxPool2d((None, 7))
input = torch.randn(1, 64, 10, 9)
output = m(input)
# 输出大小为 1x1
m = nn.AdaptiveMaxPool2d((1))
input = torch.randn(1, 64, 10, 9)
output = m(input)
print(output.size())

torch.Size([1, 64, 1, 1])
